In [14]:
from IPython import get_ipython
from IPython.display import clear_output, display

ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_108/1711627342.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_108/1711627342.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [15]:
import os
from typing import List, Optional, Union, Dict, Tuple
from pathlib import Path 

import torch
from torch import Tensor
import numpy as np
import einops
from fancy_einsum import einsum
import circuitsvis as cv

import transformer_lens.utils as tl_utils

from transformer_lens import HookedTransformer
import transformer_lens.patching as patching

from transformers import AutoModelForCausalLM

from torch import Tensor
from jaxtyping import Float
import plotly.express as px

from functools import partial

from torchtyping import TensorType as TT

from path_patching_cm.path_patching import Node, IterNode, path_patch, act_patch
from path_patching_cm.ioi_dataset import IOIDataset, NAMES
from neel_plotly import imshow as imshow_n

from utils.visualization import imshow_p, plot_attention_heads, plot_attention

from utils.visualization_utils import (
    plot_attention_heads,
    scatter_attention_and_contribution,
    get_attn_head_patterns
)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [17]:
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-160m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
)
model.set_use_hook_mlp_in(True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


## Greater-Than

In [16]:
from data.greater_than_dataset import get_prob_diff, YearDataset, get_valid_years

In [26]:
ds = YearDataset(get_valid_years(model.tokenizer, 1100, 1800), 1000, Path("data/potential_nouns.txt"), model.tokenizer)

def batch(iterable, n:int=1):
   current_batch = []
   for item in iterable:
       current_batch.append(item)
       if len(current_batch) == n:
           yield current_batch
           current_batch = []
   if current_batch:
       yield current_batch

clean = list(batch(ds.good_sentences, 9))
labels = list(batch(ds.years_YY, 9))
corrupted = list(batch(ds.bad_sentences, 9))



In [27]:
clean[0], labels[0], corrupted[0]

(['The shift lasted from the year 1702 to the year 17',
  'The growth lasted from the year 1703 to the year 17',
  'The program lasted from the year 1704 to the year 17',
  'The deal lasted from the year 1705 to the year 17',
  'The employment lasted from the year 1706 to the year 17',
  'The shift lasted from the year 1707 to the year 17',
  'The order lasted from the year 1708 to the year 17',
  'The impact lasted from the year 1709 to the year 17',
  'The challenge lasted from the year 1710 to the year 17'],
 [tensor(2),
  tensor(3),
  tensor(4),
  tensor(5),
  tensor(6),
  tensor(7),
  tensor(8),
  tensor(9),
  tensor(10)],
 ['The shift lasted from the year 1701 to the year 17',
  'The growth lasted from the year 1701 to the year 17',
  'The program lasted from the year 1701 to the year 17',
  'The deal lasted from the year 1701 to the year 17',
  'The employment lasted from the year 1701 to the year 17',
  'The shift lasted from the year 1701 to the year 17',
  'The order lasted f

In [31]:
model.to_tokens([str(x) for x in range(10, 100)], prepend_bos=False)

tensor([[ 740],
        [ 883],
        [ 805],
        [1012],
        [1047],
        [1010],
        [1036],
        [1166],
        [1093],
        [ 746],
        [ 938],
        [1797],
        [1423],
        [1508],
        [1348],
        [1099],
        [1731],
        [1630],
        [1619],
        [1717],
        [1229],
        [2405],
        [1237],
        [1610],
        [1706],
        [1671],
        [1812],
        [1787],
        [1839],
        [1867],
        [1449],
        [3156],
        [2945],
        [3079],
        [2031],
        [1857],
        [2950],
        [2504],
        [2385],
        [2537],
        [1235],
        [3712],
        [3583],
        [3357],
        [3439],
        [2417],
        [3208],
        [3011],
        [3680],
        [3046],
        [1549],
        [3832],
        [3763],
        [3571],
        [1540],
        [2082],
        [2526],
        [2251],
        [2358],
        [2090],
        [1967],
        [3677],
        

In [28]:
input_length = 1 + len(model.tokenizer(ds.good_sentences[0])[0])
prob_diff = get_prob_diff(model.tokenizer)

In [29]:
prob_diff.

<function data.greater_than_dataset.get_prob_diff.<locals>.prob_diff(logits, years)>